In [1]:
# echo-server.py
# import selectors
# from _thread import *
# import threading
import socket
import time
from random import randint
import json 
import uuid
import os

'''
Program variables
'''
LAN_IP = "192.168.1.255"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)
EVENT_TIMEOUT = 5000 #ms #time before an emergency event times out regardless of acks
ACK_TIMEOUT = 200 #ms time before emergency message is broadcasted to un-acked vehicles
PROXIMITY_PURGE_TIME = 5000 #ms
PROXIMITY_THRESHOLD = 10 # 1 dimensional coordinate, meters

#unused function. May be used for threading/user io
def thread(Client, Pserver):
    while True:
        # data received from client
        data = Pserver.recv(1024)
        if data:
            Client.send(data)
        elif not data:
            print_lock.release()
            break

'''
Vehicle class
Does the buld of the heavy lifting for
variable storage, send/recieve routines, variable updates and maintanance
Takes in a defined IP and port number if none-default values are desired
'''
class Vehicle:
    def __init__(self, LAN_IP="127.0.0.1", PORT=9999):
        self.location = 0 # 1 dimensional x coordinate
        self.last_update = time.time()
        self.msg = ""
        self.buffer = ""
        self.proximity_table = {}
        self.velocity = 1 # m/s
        self.LAN_IP = LAN_IP
        self.PORT = PORT
        self.id = str(uuid.uuid1())
        #self.broadcast = 
        try:
            self.rx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.tx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.tx.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1) #set broadcast
            self.rx.bind((LAN_IP, PORT))
            self.rx.settimeout(1) #set timeout on socket to 1s. Val to be lowered for better latency
        except Exception as e:
            print(e) 
    
    def create_transmit_socket(self):
        init_socket = None
        try:
            init_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            init_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1) #set broadcast
        except Exception as e:
            print(e) 
        return init_socket
    
    
    def update_location(self):
        current_time = time.time()
        self.location = self.location + self.velocity * (current_time - self.last_update)
        self.last_update = current_time
        return self.location

    
    def broadcast_location(self):
        try:
            self.update_location()
            broadcast_loc_msg = {"msg_type": "location_update", "time": time.time(), "vehicle_id": self.id, 
            "location": str(self.location)}
            self.tx.sendto(json.dumps(broadcast_loc_msg).encode(), (LAN_IP, PORT))
        except Exception as e:
            print(e) 
        return "location sent"
        

    def send_event_msg(self, msg):
        try:
            self.tx.sendto(msg, (LAN_IP, PORT))
            print("request sent")
        except Exception as e:
            print(e)
        return "sent event msg"
    
    
    def recieve(self):
        try:
            self.recieve_data = self.rx.recv(1024)
            if self.recieve_data:
                print("recieved reply")
                return self.recieve_data
            elif not self.recieve_data:
                return ''
        except Exception as e:
            print(e)
            return ''
    
    
    def update_proximity_table(self, parsed_dic):
        self.update_location()
        proximity = abs(float(parsed_dic["location"]) - float(self.location))
        vehicle_id = parsed_dic["vehicle_id"]
        
        if vehicle_id != self.id and proximity < PROXIMITY_THRESHOLD:
            self.proximity_table[vehicle_id] = time.time()
            print("updating  proximity table")
    
    
    def clean_proximity_table(self):
        for key, val in self.proximity_table.items():
            current_time = time.time()
            if(val - current_time) > PROXIMITY_PURGE_TIME:
                print("Deleting table v instance")
                try:
                    del self.proximity_table[key]
                except Exception as e:
                    print(e)
    
    
    def parse(self, data_dict):
        if data_dict["msg_type"] == "location_update":
            self.update_proximity_table(data_dict)
        elif data_dict["msg_type"] == "crash":
            if self.id in data_dict["proximity_table"]:
                self.send_ack()
    
    
    def send_ack(self):
        ack = json.dumps({"msg_type": "ack", "vehicle_id": self.id}).encode()
        self.tx.sendto(ack, (LAN_IP, PORT))


#------------------------------------------------------------------------------------- end of Vehicle class



def emergency_event(vehicle):
    # initial variables for vehicles we want acks from 
    # and timeout value
    ack_table = vehicle.proximity_table.copy()
    current_time = time.time()
    
    #Keep sending emergency requests until we have all of the acks we want or we hit a timeout
    while ack_table:
        emergency_msg = {"msg_type": "crash", "time": current_time, "vehicle.id": vehicle.id, 
            "proximity_table": ack_table}
        vehicle.send_event_msg(json.dumps(emergency_msg).encode())
        
        #Listen for messages and remove ack ids from table as they come in
        while (time.time() - current_time) < ACK_TIMEOUT:
            data = vehicle.recieve()
            if data:
                data_dic = json.loads(data.decode())
                if data_dic["msg_type"] == "ack":
                    print("recieved data in event fxn")
                    ack_id = data_dic["vehicle_id"]
                    if ack_id in ack_table:
                        try:
                            del ack_table[ack_id]
                            print("ack recieved and removed from proximity table")
                        except Exception as e:
                            print("failed to remove id from event proximity table")
                            print(e)
            #check for message timeout and quite if it's exceeeded
            if (time.time() - current_time) > EVENT_TIMEOUT:
                return "Failed to recieve acks from vehicles"
        time.sleep(.2) #back off for 10ms to let acks come in etc
    return "recieved acks for all vehicles"

In [2]:
vehicle = Vehicle(LAN_IP=LAN_IP)
n = 1000
s_per_crash = 30 #loops take >~1second, if a crash is desired at E[p] seconds, then seconds = n*p loops
#in other words, 1 / x loops should be positive
p = s_per_crash * 10
while True:
    vehicle.broadcast_location()
    vehicle.clean_proximity_table()
    data = vehicle.recieve()
    
    if data:
        data_dict = json.loads(data.decode())
        msg_type = vehicle.parse(data_dict)
    if randint(0, n) < p: #should run every s_per_crash loops
        print("emergency event triggered")
        emergency_event(vehicle)
        
    time.sleep(1)

recieved reply
emergency event triggered
recieved reply
updating  proximity table
recieved reply
recieved reply
updating  proximity table
recieved reply
recieved reply
updating  proximity table
recieved reply
recieved reply
emergency event triggered
request sent
recieved reply
recieved reply
recieved reply
recieved reply
recieved data in event fxn
recieved reply
recieved reply
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out
timed out


In [81]:
randint(0, 10000)

424

In [4]:
MESSAGE = b"Hello, World!"
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

In [6]:
sock.sendto(MESSAGE, (LAN_IP, PORT))

13

In [21]:
sock.bind((LAN_IP, PORT))
while True:
    data, addr = sock.recvfrom(1024)
    print("recieved msg %s" %data)

recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World! from tm1'


In [8]:
vehicle = Vehicle(LAN_IP=LAN_IP)

NameError: name 'Vehicle' is not defined

In [9]:
vehicle.broadcast_location()

NameError: name 'vehicle' is not defined

In [ ]:
if __name__ == "_main__":
    vehicle = Vehicle(LAN_IP= LAN_IP)
    
    while True:
        vehicle.broadcast()

        vehicle.recieve()
        if vehicle.recieve_data:
            #parse data

            #else is event:
                #if parse for ID: send ack
        
        #Initiate emergency message
        #if randint(0,1000) > 500:
        
            break

In [45]:
msg = {"msg_type": "location_update", "time": time.time(), "vehicle_id": "12345", 
            "location": 70}
vehicle.update_proximity_table(msg)

updating  proximity table


In [55]:
p_copy = vehicle.proximity_table.copy()

In [49]:
vehicle.location

71.82640838623047

In [62]:
vehicle.proximity_table

{'12345': 1670148297.1142504}

In [58]:
p_copy["test00"] = 0

In [61]:
p_copy

{'12345': 1670148297.1142504, 'test00': 0}

In [77]:
json.dumps({"tst": str(uuid.uuid1())})

'{"tst": "d1fd6908-7208-11ed-8700-a7057b74caa5"}'